## Chapter 8

## Feature Importance

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier
from sklearn.metrics import roc_curve, classification_report, confusion_matrix, log_loss, accuracy_score
from sklearn.datasets import make_classification
import time

#### Chapter 8 - code snippets

In [2]:
# SNIPPET 8.7 CREATING A SYNTHETIC DATASET

def getTestData(n_features=40,n_informative=10,n_redundant=10,n_samples=10000):
    
    # generate a random dataset for a classification problem
    from sklearn.datasets import make_classification
    
    trnsX,cont=make_classification(n_samples=n_samples,n_features=n_features,n_informative=n_informative,
                                   n_redundant=n_redundant,random_state=0, shuffle=False)
    
    df0=pd.DatetimeIndex(periods=n_samples,freq=pd.tseries.offsets.BDay(), end=pd.datetime.today())
    
    trnsX,cont=pd.DataFrame(trnsX,index=df0),pd.Series(cont,index=df0).to_frame('bin')
    
    df0=['I_'+str(i) for i in range(n_informative)]+['R_'+str(i) for i in range(n_redundant)]
    
    df0+=['N_'+str(i) for i in range(n_features-len(df0))]
    
    trnsX.columns=df0
    
    cont['w']=1./cont.shape[0]
    cont['t1']=pd.Series(cont.index,index=cont.index)
    
    return trnsX,cont



#-----------------------------------------------------------------------------------------------

# This version is for large n_samples. It changes the frequency of the index of the dataset.

def getTestData_largeSet(n_features=40,n_informative=10,n_redundant=10,n_samples=10000):
    
    # generate a random dataset for a classification problem
    from sklearn.datasets import make_classification
    
    trnsX,cont=make_classification(n_samples=n_samples,n_features=n_features,n_informative=n_informative,
                                   n_redundant=n_redundant,random_state=0, shuffle=False)
    
    cond = 0
    
    if (n_samples <= 1000F0):       
        df0=pd.DatetimeIndex(periods=n_samples,freq=pd.tseries.offsets.BDay(), end=pd.datetime.today())      # original
        
    elif (n_samples>10000) & (n_samples<=100000):
        df0=pd.DatetimeIndex(periods=n_samples,freq=pd.tseries.offsets.Minute(60), end=pd.datetime.today())   #...mc
        
    elif (n_samples>100000) & (n_samples<=400000):
        df0=pd.DatetimeIndex(periods=n_samples,freq=pd.tseries.offsets.Minute(30), end=pd.datetime.today())   #...mc
        
    else:
        df0=pd.DatetimeIndex(periods=n_samples,freq=pd.tseries.offsets.Minute(15), end=pd.datetime.today())   #...mc
        
                
        
    trnsX,cont=pd.DataFrame(trnsX,index=df0),pd.Series(cont,index=df0).to_frame('bin')
    
    df0=['I_'+str(i) for i in range(n_informative)]+['R_'+str(i) for i in range(n_redundant)]
    
    df0+=['N_'+str(i) for i in range(n_features-len(df0))]
    
    trnsX.columns=df0
    cont['w']=1./cont.shape[0]
    cont['t1']=pd.Series(cont.index,index=cont.index)
    
    return trnsX,cont


In [3]:
#SNIPPET 8.5 COMPUTATION OF ORTHOGONAL FEATURES

def get_eVec(dot,varThres):
   
    # compute eVec from dot prod matrix, reduce dimension
    eVal,eVec=np.linalg.eigh(dot)
    idx=eVal.argsort()[::-1] # arguments for sorting eVal desc
    eVal,eVec=eVal[idx],eVec[:,idx]
    
    #2) only positive eVals
    eVal=pd.Series(eVal,index=['PC_'+str(i+1) for i in range(eVal.shape[0])])
    eVec=pd.DataFrame(eVec,index=dot.index,columns=eVal.index)
    eVec=eVec.loc[:,eVal.index]
    
    #3) reduce dimension, form PCs
    cumVar=eVal.cumsum()/eVal.sum()
    dim=cumVar.values.searchsorted(varThres)
    eVal,eVec=eVal.iloc[:dim+1],eVec.iloc[:,:dim+1]
    
    return eVal,eVec
#-----------------------------------------------------------------

def orthoFeats(dfX,varThres=.95):
   
    # Given a dataframe dfX of features, compute orthofeatures dfP
    dfZ=dfX.sub(dfX.mean(),axis=1).div(dfX.std(),axis=1) # standardize
    dot=pd.DataFrame(np.dot(dfZ.T,dfZ),index=dfX.columns,columns=dfX.columns)       # esta matriz es multiplo de la matriz de correlación
    eVal,eVec=get_eVec(dot,varThres)
    
    #dfP=np.dot(dfZ,eVec)                #original
    dfP = pd.DataFrame(np.dot(dfZ,eVec), index=dfX.index) 
     
    return dfP

In [4]:
# SNIPPET 8.2 MDI FEATURE IMPORTANCE

def featImpMDI(fit,featNames):
    
    # feat importance based on IS mean impurity reduction
    df0={i:tree.feature_importances_ for i,tree in enumerate(fit.estimators_)}
    df0=pd.DataFrame.from_dict(df0,orient='index')
    df0.columns=featNames
    df0=df0.replace(0,np.nan) # because max_features=1
    imp=pd.concat({'mean':df0.mean(),'std':df0.std()*df0.shape[0]**-.5},axis=1)
    imp/=imp['mean'].sum()
    
    return imp

In [5]:
# SNIPPET 8.3 MDA FEATURE IMPORTANCE

def featImpMDA(clf,X,y,cv,sample_weight,t1,pctEmbargo,scoring='neg_log_loss'):
    
    # feat importance based on OOS score reduction
    
    if scoring not in ['neg_log_loss','accuracy']:
        raise Exception('wrong scoring method.')
    from sklearn.metrics import log_loss,accuracy_score
    
    cvGen=PurgedKFold(n_splits=cv,t1=t1,pctEmbargo=pctEmbargo) # purged cv
    scr0,scr1=pd.Series(),pd.DataFrame(columns=X.columns)
    
    for i,(train,test) in enumerate(cvGen.split(X=X)):
        X0,y0,w0=X.iloc[train,:],y.iloc[train],sample_weight.iloc[train]
        X1,y1,w1=X.iloc[test,:],y.iloc[test],sample_weight.iloc[test]
        fit=clf.fit(X=X0,y=y0,sample_weight=w0.values)
        
        if scoring=='neg_log_loss':
            prob=fit.predict_proba(X1)
            scr0.loc[i]=-log_loss(y1,prob,sample_weight=w1.values,labels=clf.classes_)
        else:
            pred=fit.predict(X1)
            scr0.loc[i]=accuracy_score(y1,pred,sample_weight=w1.values)
        
        for j in X.columns:
            X1_=X1.copy(deep=True)
            np.random.shuffle(X1_[j].values) # permutation of a single column
            
            if scoring=='neg_log_loss':
                prob=fit.predict_proba(X1_)
                scr1.loc[i,j]=-log_loss(y1,prob,sample_weight=w1.values,labels=clf.classes_)
            else:
                pred=fit.predict(X1_)
                scr1.loc[i,j]=accuracy_score(y1,pred,sample_weight=w1.values)
    
    imp=(-scr1).add(scr0,axis=0)
    
    if scoring=='neg_log_loss':
        imp=imp/-scr1
    else:
        imp=imp/(1.-scr1)
    
    imp=pd.concat({'mean':imp.mean(),'std':imp.std()*imp.shape[0]**-.5},axis=1)
    
    return imp,scr0.mean()


In [6]:
# SNIPPET 8.4 IMPLEMENTATION OF SFI

def auxFeatImpSFI(featNames,clf,trnsX,cont,scoring,cvGen):
    
    imp=pd.DataFrame(columns=['mean','std'])
    
    for featName in featNames:
        
        df0=cvScore(clf,X=trnsX[[featName]],y=cont['bin'],sample_weight=cont['w'],
                    scoring=scoring,cvGen=cvGen)
        imp.loc[featName,'mean']=df0.mean()
        imp.loc[featName,'std']=df0.std()*df0.shape[0]**-.5
        
    return imp

In [7]:
# SNIPPET 8.8 CALLING FEATURE IMPORTANCE FOR ANY METHOD

def featImportance(trnsX,cont,n_estimators=1000,cv=10,max_samples=1.,numThreads=24,
                   pctEmbargo=0,scoring='accuracy',method='SFI',minWLeaf=0.,**kargs):
    
    # feature importance from a random forest
    from sklearn.tree import DecisionTreeClassifier
    from sklearn.ensemble import BaggingClassifier
    from mpEngine import mpPandasObj
    
    n_jobs=(-1 if numThreads>1 else 1) # run 1 thread with ht_helper in dirac1
    
    #1) prepare classifier,cv. max_features=1, to prevent masking
    clf=DecisionTreeClassifier(criterion='entropy',max_features=1,
    class_weight='balanced',min_weight_fraction_leaf=minWLeaf)
    clf=BaggingClassifier(base_estimator=clf,n_estimators=n_estimators,
    max_features=1.,max_samples=max_samples,oob_score=True,n_jobs=n_jobs)
    fit=clf.fit(X=trnsX,y=cont['bin'],sample_weight=cont['w'].values)
    oob=fit.oob_score_
    
    if method=='MDI':
        imp=featImpMDI(fit,featNames=trnsX.columns)
        oos=cvScore(clf,X=trnsX,y=cont['bin'],cv=cv,sample_weight=cont['w'],
                    t1=cont['t1'],pctEmbargo=pctEmbargo,scoring=scoring).mean()
        
    elif method=='MDA':
        imp,oos=featImpMDA(clf,X=trnsX,y=cont['bin'],cv=cv,sample_weight=cont['w'],
                           t1=cont['t1'],pctEmbargo=pctEmbargo,scoring=scoring)
    
    elif method=='SFI':
        cvGen=PurgedKFold(n_splits=cv,t1=cont['t1'],pctEmbargo=pctEmbargo)
        oos=cvScore(clf,X=trnsX,y=cont['bin'],sample_weight=cont['w'],scoring=scoring,
                    cvGen=cvGen).mean()
        clf.n_jobs=1 # paralellize auxFeatImpSFI rather than clf
        imp=mpPandasObj(auxFeatImpSFI,('featNames',trnsX.columns),numThreads,
                        clf=clf,trnsX=trnsX,cont=cont,scoring=scoring,cvGen=cvGen)
        
    return imp,oob,oos


# ----------------------------------------------------------------------------------------

def featImportance_2_mc(trnsX,cont,n_estimators=1000,cv=10,max_samples=1.,
                   pctEmbargo=0,scoring='accuracy',method='SFI',minWLeaf=0.,**kargs):
    
    # feature importance from a random forest
    from sklearn.tree import DecisionTreeClassifier
    from sklearn.ensemble import BaggingClassifier
    #from mpEngine import mpPandasObj                  #mc
    
    # n_jobs=(-1 if numThreads>1 else 1) # run 1 thread with ht_helper in dirac1    ...#mc
    
    #1) prepare classifier,cv. max_features=1, to prevent masking
    clf=DecisionTreeClassifier(criterion='entropy',max_features=1,class_weight='balanced',
                               min_weight_fraction_leaf=minWLeaf)
    #clf=BaggingClassifier(base_estimator=clf,n_estimators=n_estimators,            ...#mc
    #                        max_features=1.,max_samples=max_samples,oob_score=True,n_jobs=n_jobs)
    
    clf=BaggingClassifier(base_estimator=clf,n_estimators=n_estimators,max_features=1.,
                          max_samples=max_samples,oob_score=True)
    fit=clf.fit(X=trnsX,y=cont['bin'],sample_weight=cont['w'].values)
    oob=fit.oob_score_
    
    if method=='MDI':
        imp=featImpMDI(fit,featNames=trnsX.columns)
        
        #oos=cvScore(clf,X=trnsX,y=cont['bin'],cv=cv,sample_weight=cont['w'],                     #original
        #            t1=cont['t1'],pctEmbargo=pctEmbargo,scoring=scoring).mean()
        
        oos = cvScore2_mc(clf,X=trnsX,y=cont['bin'],cv=cv,sample_weight=cont['w'],                # mc
                          t1=cont['t1'],pctEmbargo=pctEmbargo,scoring=scoring).mean()
              
    elif method=='MDA':
        imp,oos=featImpMDA(clf,X=trnsX,y=cont['bin'],cv=cv,sample_weight=cont['w'],
                           t1=cont['t1'],pctEmbargo=pctEmbargo,scoring=scoring)
    
    elif method=='SFI':
        cvGen=PurgedKFold(n_splits=cv,t1=cont['t1'],pctEmbargo=pctEmbargo)
        
        #oos=cvScore(clf,X=trnsX,y=cont['bin'],sample_weight=cont['w'],scoring=scoring,           #original
        #            cvGen=cvGen).mean()
        
        oos = cvScore2_mc(clf, X=trnsX,y=cont['bin'],sample_weight=cont['w'],scoring=scoring,     #...mc
                                cvGen=cvGen).mean()
        
        #clf.n_jobs=1 # paralellize auxFeatImpSFI rather than clf                                 # original
        #imp=mpPandasObj(auxFeatImpSFI,('featNames',trnsX.columns),numThreads,
        #                clf=clf,trnsX=trnsX,cont=cont,scoring=scoring,cvGen=cvGen)
        
        
        imp = auxFeatImpSFI(featNames=trnsX.columns, clf=clf, trnsX=trnsX, cont=cont, scoring=scoring, cvGen=cvGen)     #mc
        
        
    return imp,oob,oos

In [8]:
#SNIPPET 7.3 CROSS-VALIDATION CLASS WHEN OBSERVATIONS OVERLAP

from sklearn.model_selection._split import _BaseKFold

class PurgedKFold(_BaseKFold):
    '''
    Extend KFold class to work with labels that span intervals
    The train is purged of observations overlapping test-label intervals
    Test set is assumed contiguous (shuffle=False), w/o training samples in between
    '''
    
    def __init__(self, n_splits=3, t1=None, pctEmbargo=0.):
        
        if not isinstance(t1, pd.Series):
            raise ValueError('Label Through Dates must be a pd.Series')
            
        super(PurgedKFold,self).__init__(n_splits,shuffle=False,random_state=None)
        self.t1=t1
        self.pctEmbargo=pctEmbargo
    
    
    def split(self,X,y=None,groups=None):
        
        if (X.index==self.t1.index).sum()!=len(self.t1):
            raise ValueError('X and ThruDateValues must have the same index')
            
        indices=np.arange(X.shape[0])
        mbrg=int(X.shape[0]*self.pctEmbargo)
        test_starts=[(i[0],i[-1]+1) for i in \
            np.array_split(np.arange(X.shape[0]),self.n_splits)]
        
        for i,j in test_starts:
            
            t0=self.t1.index[i] # start of test set
            test_indices=indices[i:j]
            maxT1Idx=self.t1.index.searchsorted(self.t1[test_indices].max())
            train_indices=self.t1.index.searchsorted(self.t1[self.t1<=t0].index)
            
            if maxT1Idx<X.shape[0]: # right train (with embargo)
                train_indices=np.concatenate((train_indices,indices[maxT1Idx+mbrg:]))
                
            yield train_indices,test_indices

In [9]:
#SNIPPET 7.4 USING THE PurgedKFold CLASS

def cvScore(clf, X, y, sample_weight, scoring='neg_log_loss', t1=None, cv=None, cvGen=None, pctEmbargo=None):
    
    if scoring not in ['neg_log_loss','accuracy']:
        raise Exception('wrong scoring method.')
    
    from sklearn.metrics import log_loss,accuracy_score
    #from clfSequential import PurgedKFold
    
    
    if cvGen is None:
        
        cvGen=PurgedKFold(n_splits=cv,t1=t1,pctEmbargo=pctEmbargo) # purged
    
    '''
    if cvGen is None:
        if pctEmbargo==None:
            cvGen = PurgedKFold(n_splits=cv, t1=t1)
    else:
            cvGen=PurgedKFold(n_splits=cv,t1=t1,pctEmbargo=pctEmbargo) # purged
    '''
    
    score=[]
    
    for train,test in cvGen.split(X=X):    # book
            
        fit=clf.fit(X=X.iloc[train,:],y=y.iloc[train],sample_weight=sample_weight.iloc[train].values) # book
        
        #fit=clf.fit(X=X.iloc[train,:],y=y.iloc[train].values.reshape(-1),\
        #            sample_weight=sample_weight.iloc[train].values.reshape(-1))   # mc
        
        
        if scoring=='neg_log_loss':
            prob=fit.predict_proba(X.iloc[test,:])
            score_=-log_loss(y.iloc[test],prob,sample_weight=sample_weight.iloc[test].values,labels=clf.classes_)
        else:
            pred=fit.predict(X.iloc[test,:])
            score_=accuracy_score(y.iloc[test],pred,sample_weight= \
                sample_weight.iloc[test].values)
            
        score.append(score_)
        
    return np.array(score)


# ----------------------- --------------------- ------------------------- ---------------------------


def cvScore2_mc(clf, X, y, sample_weight, scoring='neg_log_loss', t1=None, cv=None, cvGen=None, pctEmbargo=None):
    
    if scoring not in ['neg_log_loss','accuracy']:
        raise Exception('wrong scoring method.')
    
    from sklearn.metrics import log_loss,accuracy_score
    #from clfSequential import PurgedKFold
    
    if cvGen is None:                                   # mc
        if pctEmbargo==None:                            # mc
            cvGen = PurgedKFold(n_splits=cv, t1=t1)     # mc
        else:
            cvGen=PurgedKFold(n_splits=cv,t1=t1,pctEmbargo=pctEmbargo) # purged , book
    
    score=[]
    
    for train,test in cvGen.split(X=X):    
            
        #fit=clf.fit(X=X.iloc[train,:],y=y.iloc[train],sample_weight=sample_weight.iloc[train].values) # book
        
        fit=clf.fit(X=X.iloc[train,:],y=y.iloc[train].values.reshape(-1),\
                    sample_weight=sample_weight.iloc[train].values.reshape(-1))   # mc
        
        
        
        
        if scoring=='neg_log_loss':
            
            prob=fit.predict_proba(X.iloc[test,:])
            
            #score_=-log_loss(y.iloc[test],prob,sample_weight=sample_weight.iloc[test].values,labels=clf.classes_)
            score_=-log_loss(y.iloc[test],prob,sample_weight=sample_weight.iloc[test].values.reshape(-1),labels=clf.classes_)
            
        else:
            pred=fit.predict(X.iloc[test,:])
            score_=accuracy_score(y.iloc[test],pred,sample_weight= \
                sample_weight.iloc[test].values)
            
        score.append(score_)
        
    return np.array(score)
  


**Plotting Function**

In [10]:
def plotFeatImportance(pathOut,imp,oob,oos,method,tag=0,simNum=0,**kargs):
    
    # plot mean imp bars with std
    mpl.figure(figsize=(10,imp.shape[0]/5.))
    imp=imp.sort_values('mean',ascending=True)
    ax=imp['mean'].plot(kind='barh',color='b',alpha=.25,xerr=imp['std'],
                        error_kw={'ecolor':'r'})
    
    if method=='MDI':
        mpl.xlim([0,imp.sum(axis=1).max()])
        mpl.axvline(1./imp.shape[0],linewidth=1,color='r',linestyle='dotted')
    
    ax.get_yaxis().set_visible(False)
    
    for i,j in zip(ax.patches,imp.index):
        ax.text(i.get_width()/2,i.get_y()+i.get_height()/2,j,ha='center',va='center',color='black')
    
    mpl.title('tag='+tag+' | simNum='+str(simNum)+' | oob='+str(round(oob,4))+ ' | oos='+str(round(oos,4)))
    mpl.savefig(pathOut+'featImportance_'+str(simNum)+'.png',dpi=100)
    mpl.clf();mpl.close()
    
    return

#### Exercises:

**8.1) Using the code in Section 8.6:**

**(a) Generate a dataset (X, y).**

In [11]:
trns_X, trns_y =  getTestData()

C:\Users\ASUS\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  # This is added back by InteractiveShellApp.init_path()


TypeError: __new__() got an unexpected keyword argument 'periods'

In [ ]:
trns_X.head()

In [ ]:
trns_y.head()

**(b) Apply a PCA transformation on X, wich we donote $\dot{X}$.**

In [ ]:
X_p = orthoFeats(trns_X, varThres = 0.95)

In [ ]:
X_p.shape

In [ ]:
X_p.head()

In [ ]:
trns_X.shape

In [ ]:
X_p.shape

**(c) Compute MDI, MDA, and SFI feature importance on ($\dot{X}$,y), where the base estimator is RF.**

In [ ]:
clf = RandomForestClassifier(n_estimators = 1000, class_weight = 'balanced_subsample', criterion='entropy')

In [ ]:
X_p.head()

In [ ]:
trns_X.head()

In [ ]:
trns_y.head()

**MDI:**

In [ ]:
cv = 10
pctEmbargo = 0
max_samples = 1.

In [ ]:
calculate_MDI = False

if calculate_MDI:

    start = time.time()

    MDI_imp, MDI_oob, MDI_oos = featImportance_2_mc(trnsX=X_p, cont= trns_y, n_estimators=1000, cv=cv, max_samples=max_samples,
                       pctEmbargo=pctEmbargo, scoring='accuracy', method='MDI', minWLeaf=0.)

    end = time.time()

    print(f'MDI takes {(end-start)/60} minutes')

**MDA:**

In [ ]:
calculate_MDA = False

if calculate_MDA:

    start = time.time()

    MDA_imp, MDA_oob, MDA_oos = featImportance_2_mc(trnsX=X_p, cont= trns_y, n_estimators=1000, cv=10, max_samples=1.,
                       pctEmbargo=0, scoring='accuracy', method='MDA', minWLeaf=0.)

    end = time.time() 
    print(f'MDA takes {(end-start)/60} minutes')

**SFI:**

In [ ]:
calculate_SFI = False

if calculate_SFI:
    
    start = time.time()

    SFI_imp, SFI_oob, SFI_oos = featImportance_2_mc(trnsX=X_p, cont= trns_y, n_estimators=1000, cv=10, max_samples=1.,
                       pctEmbargo=0, scoring='accuracy', method='SFI', minWLeaf=0.)

    end = time.time()
    print((end-start)/60)
    print(f'SFI takes {(end-start)/60/60} hours')

**Calling the values MDI, MDA and SFI results previously saved:**

In [ ]:
MDI_imp = pd.read_csv('MDI_imp.csv')
MDI_imp = MDI_imp.drop(['Unnamed: 0'], axis=1)
MDI_oob_oos = pd.read_csv('MDI_oob_oos.csv')

MDA_imp = pd.read_csv('MDA_imp.csv')
MDA_imp = MDA_imp.drop(['Unnamed: 0'], axis=1)
MDA_oob_oos = pd.read_csv('MDA_oob_oos.csv')

SFI_imp = pd.read_csv('SFI_imp.csv')
SFI_imp = SFI_imp.drop(['Unnamed: 0'], axis=1)
SFI_oob_oos = pd.read_csv('SFI_oob_oos.csv')


In [ ]:
MDI_oob = MDI_oob_oos.iloc[0,1]
MDI_oos = MDI_oob_oos.iloc[0,2]

print(f'MDI_oob = {MDI_oob}')
print(f'MDI_oos= {MDI_oos}')

In [ ]:
MDA_oob = MDA_oob_oos.iloc[0,1]
MDA_oos = MDA_oob_oos.iloc[0,2]

print(f'MDA_oob = {MDA_oob}')
print(f'MDA_oos= {MDA_oos}')

In [ ]:
SFI_oob = SFI_oob_oos.iloc[0,1]
SFI_oos = SFI_oob_oos.iloc[0,2]

print(f'SFI_oob = {SFI_oob}')
print(f'SFI_oos= {SFI_oos}')

**(d) Do the three methods agree on what features are important? why?**

In [ ]:
MDI_imp_sorted = MDI_imp.sort_values(by=['mean'], ascending=False)
MDA_imp_sorted = MDA_imp.sort_values(by=['mean'], ascending=False)
SFI_imp_sorted = SFI_imp.sort_values(by=['mean'], ascending=False)

In [ ]:
MDI_imp_sorted.head()

In [ ]:
MDA_imp_sorted.head(5)

In [ ]:
features_imp = (pd.DataFrame()
               .assign(MDI_components = MDI_imp_sorted.index)
               .assign(MDI = MDI_imp_sorted[['mean']].values)
               .assign(MDA_components = MDA_imp_sorted.index)
               .assign(MDA = MDA_imp_sorted[['mean']].values)
               .assign(SFI_components = SFI_imp_sorted.index)
               .assign(SFI = SFI_imp_sorted[['mean']].values))

In [ ]:
features_imp.head(10)

 The three methods agree on the selection of the first five components.
 
The MDI and MDA methods extend their selection coincidence up to the ninth component but in different order. This could be due to the orthogonalization of the features through PCA previously applied to the original dataset, which partially solved the substitution effect. For the SFI method substitution effects do not take place and its results may not match completely bacause it computes the OOS performance score of each feature in isolation, nevertheless SFI selected first ten components coincide mostly with MDI and MDA selected components.

**MDI_imp calculation changing the columns order of X_p (PCA) to verify if the selected features match.**

In [ ]:
cols = []
for c in reversed(X_p.columns):
    cols.append(c)

Xp2 = X_p[cols]

In [ ]:
Xp2.head(3)

In [ ]:
calculate_MDI = False

if calculate_MDI:

    start = time.time()

    MDI_imp_Xp2, MDI_oob_Xp2, MDI_oos_Xp2 = featImportance_2_mc(trnsX=X_p2, cont= trns_y, n_estimators=1000, cv=cv, max_samples=max_samples,
                       pctEmbargo=pctEmbargo, scoring='accuracy', method='MDI', minWLeaf=0.)

    end = time.time()

    print(f'MDI takes {(end-start)/60} minutes')

In [ ]:
if calculate_MDI:
    MDI_imp_df = (pd.DataFrame()
                  .assign(PCA_Xp = MDI_imp_sorted.index)
                  .assign(MDI_imp_Xp = MDI_imp_sorted['mean'].values)
                  .assign(PCA_Xp2 = MDI_imp_Xp2.sort_values(by = ['mean'], ascending = False).index)
                  .assign(MDI_imp_Xp2 = MDI_imp_Xp2.sort_values(by=['mean'], ascending= False)[['mean']].values)
                 )
    
    MDI_imp_df.head()



Ther previous procedure consits in calculating the Principal Components of the original dataset which returns a new dataset with less features, the ones that contains the 95% of the variation of the data, 28 instead of 40 from the original dataset. Then calculate the MDI_imp to this transformed dataset, change its column order and also calculate the MDI_imp to this dataset in order to verify that in both cases the features or the selected components matches.

The result shows that the components selected in both cases are the same.


**8.2) From exercise 1, generate a new dataset ($\ddot{X},y$), where $\ddot{X}$ is a feature union ox X and $\dot{X}$.**

In [ ]:
X_pp = pd.concat([trns_X, X_p,], axis=1)

In [ ]:
X_pp.head()

In [ ]:
X_pp.shape

**8.2   a) Compute MDI, MDA, and SFI feature importance on ($\ddot{X},y)$, where the base estimator is RF.**

**MDI_Xpp:**

In [ ]:
cv = 10
pctEmbargo = 0
max_samples = 1.

In [ ]:
calculate_MDI = False

if calculate_MDI:

    start = time.time()

    MDI_imp_Xpp, MDI_oob_Xpp, MDI_oos_Xpp = featImportance_2_mc(trnsX=X_pp, cont= trns_y, n_estimators=1000, cv=cv, max_samples=max_samples,
                       pctEmbargo=pctEmbargo, scoring='accuracy', method='MDI', minWLeaf=0.)

    end = time.time()

    print(f'MDI takes {(end-start)/60} minutes')
    
else:
    MDI_imp_Xpp = pd.read_csv('MDI_imp_Xpp.csv')
    MDI_oob_oos_Xpp = pd.read_csv('MDI_oob_oos_Xpp.csv')[['MDI_oob', 'MDI_oos']]
    MDI_oob_Xpp = MDI_oob_oos_Xpp['MDI_oob'].values[0]
    MDI_oos_XXP = MDI_oob_oos_Xpp['MDI_oos'].values[0]

**MDA_Xpp:**

In [ ]:
calculate_MDA = False

if calculate_MDA:

    start = time.time()

    MDA_imp_Xpp, MDA_oob_Xpp, MDA_oos_Xpp = featImportance_2_mc(trnsX=X_pp, cont= trns_y, n_estimators=1000, cv=10, max_samples=1.,
                       pctEmbargo=0, scoring='accuracy', method='MDA', minWLeaf=0.)

    end = time.time()
    print(f'MDA takes {(end-start)/60} minutes')
    
else:
    MDA_imp_Xpp = pd.read_csv('MDA_imp_Xpp.csv')
    MDA_oob_oos_Xpp = pd.read_csv('MDA_oob_oos_Xpp.csv')[['MDA_oob', 'MDA_oos']]
    MDA_oob_Xpp = MDA_oob_oos_Xpp['MDA_oob'].values[0]
    MDA_oos_Xpp = MDA_oob_oos_Xpp['MDA_oos'].values[0]
    

**SFI_Xpp:**

In [ ]:
calculate_SFI = False

if calculate_SFI:
    
    start = time.time()

    SFI_imp_Xpp, SFI_oob_Xpp, SFI_oos_Xpp = featImportance_2_mc(trnsX=X_pp, cont= trns_y, n_estimators=1000, cv=10, max_samples=1.,
                       pctEmbargo=0, scoring='accuracy', method='SFI', minWLeaf=0.)

    end = time.time()
    print((end-start)/60)
    print(f'SFI takes {(end-start)/60/60} hours')
    
else:  
    SFI_imp_Xpp = pd.read_csv('SFI_imp_Xpp.csv')
    SFI_oob_oos_Xpp = pd.read_csv('SFI_oob_oos_Xpp.csv')[['value']]
    SFI_oob_Xpp = SFI_oob_oos_Xpp.iloc[0,0]
    SFI_oos_Xpp = SFI_oob_oos_Xpp.iloc[1,0]

In [ ]:
# Let's set the importance measures in descending order:
imp_Xpp_cols = ['features', 'mean', 'std']
MDI_imp_Xpp.columns = imp_Xpp_cols
MDA_imp_Xpp.columns = imp_Xpp_cols
SFI_imp_Xpp.columns = imp_Xpp_cols

MDI_imp_Xpp_sorted = MDI_imp_Xpp.sort_values(by=['mean'], ascending=False)
MDA_imp_Xpp_sorted = MDA_imp_Xpp.sort_values(by=['mean'], ascending=False)
SFI_imp_Xpp_sorted = SFI_imp_Xpp.sort_values(by=['mean'], ascending=False)

In [ ]:
MDI_imp_Xpp_sorted.head()

In [ ]:
MDA_imp_Xpp_sorted.head()

In [ ]:
SFI_imp_Xpp_sorted.head()

In [ ]:
features_imp_Xpp = (pd.DataFrame()
                   .assign(features_MDI = MDI_imp_Xpp_sorted['features'].values)
                   .assign(MDI_imp = MDI_imp_Xpp_sorted['mean'].values)
                   .assign(features_MDA = MDA_imp_Xpp_sorted['features'].values)
                   .assign(MDA_imp = MDA_imp_Xpp_sorted['mean'].values)
                   .assign(features_SFI = SFI_imp_Xpp_sorted['features'].values)
                   .assign(SFI_imp = SFI_imp_Xpp_sorted['mean'].values))

In [ ]:
features_imp_Xpp.head(10)

**8.2.b) Do the three methods agree on the important features? Why?**

The methods MDA and MDI do not agree on the important features due to the substitution effect but there is a high coincidence between the selected elements of the MDI and SFI methods, within the first ten selected features, MDI and SFI share 8 out of 10.

**8.3) Take the results from exercise 2:**

**8.3.a) Drop the most important features according to each method, resulting in a features matrix $\dddot{X}$.**

In [ ]:
features_imp_Xpp.head(10)

In [ ]:
MDI_topFeat_Xpp = features_imp_Xpp.features_MDI.head(10).values
MDA_topFeat_Xpp = features_imp_Xpp.features_MDA.head(10).values
SFI_topFeat_Xpp = features_imp_Xpp.features_SFI.head(10).values


print(f'MDI_topFeat_Xpp: {MDI_topFeat_Xpp}')
print(f'MDA_topFeat_Xpp: {MDA_topFeat_Xpp}')
print(f'SFI_topFeat_Xpp: {SFI_topFeat_Xpp}')

In [ ]:
MDI_MDA_SFI_topFeat_Xpp = np.concatenate((MDI_topFeat_Xpp, MDA_topFeat_Xpp, SFI_topFeat_Xpp), axis=0).tolist()
print(f'MDI_MDA_SFI_topFeat_Xpp: {MDI_MDA_SFI_topFeat_Xpp}')

# remove duplicates:
MDI_MDA_SFI_topFeat_Xpp = list(dict.fromkeys(MDI_MDA_SFI_topFeat_Xpp))
print(f'\nMDI_MDA_SFI_topFeat_Xpp: {MDI_MDA_SFI_topFeat_Xpp}')

In [ ]:
cols_Xpp = []
for c in X_pp.columns:
    cols_Xpp.append(str(c))

In [ ]:
X_pp.columns = cols_Xpp

In [ ]:
X_ppp = X_pp.drop(columns=MDI_MDA_SFI_topFeat_Xpp)

In [ ]:
X_ppp.head(3)

**8.3.b) Compute MDI, MDA, and SFI feature importance on ($\dddot{X},y$), where the base estimator is RF.**

**MDI_Xppp:**

In [ ]:
cv = 10
pctEmbargo = 0
max_samples = 1.

In [ ]:
calculate_MDI = False

if calculate_MDI:

    start = time.time()

    MDI_imp_Xppp, MDI_oob_Xppp, MDI_oos_Xppp = featImportance_2_mc(trnsX=X_ppp, cont= trns_y, n_estimators=1000, cv=cv, max_samples=max_samples,
                       pctEmbargo=pctEmbargo, scoring='accuracy', method='MDI', minWLeaf=0.)

    end = time.time()

    print(f'MDI takes {(end-start)/60} minutes')
    
else:
    MDI_imp_Xppp = pd.read_csv('MDI_imp_Xppp.csv', index_col=[0])
    MDI_oob_oos_Xppp = pd.read_csv('MDI_oob_oos_Xppp.csv',index_col=[0])
    MDI_oob_Xppp = MDI_oob_oos_Xppp.iloc[0,0]
    MDI_oos_Xppp = MDI_oob_oos_Xppp.iloc[1,0]

In [ ]:
MDI_oob_oos_Xppp = (pd.DataFrame(index=['MDI_oob', 'MDI_oos'])
                   .assign(values = pd.Series([MDI_oob_Xppp, MDI_oos_Xppp]).values))

**MDA_Xppp:**

In [ ]:
calculate_MDA = False

if calculate_MDA:

    start = time.time()

    MDA_imp_Xppp, MDA_oob_Xppp, MDA_oos_Xppp = featImportance_2_mc(trnsX=X_ppp, cont= trns_y, n_estimators=1000, cv=10, max_samples=1.,
                       pctEmbargo=0, scoring='accuracy', method='MDA', minWLeaf=0.)

    end = time.time()
    print(f'MDA takes {(end-start)/60} minutes')
    
else:
    MDA_imp_Xppp = pd.read_csv('MDA_imp_Xppp.csv', index_col=[0])
    MDA_oob_oos_Xppp = pd.read_csv('MDA_oob_oos_Xppp.csv', index_col=[0])
    MDA_oob_Xppp = MDA_oob_oos_Xppp.iloc[0,0]
    MDA_oos_Xppp = MDA_oob_oos_Xppp.iloc[1,0]
    

In [ ]:
MDA_oob_oos_Xppp

 

**SFI_Xppp:**

In [ ]:
calculate_SFI = False

if calculate_SFI:
    
    start = time.time()

    SFI_imp_Xppp, SFI_oob_Xppp, SFI_oos_Xppp = featImportance_2_mc(trnsX=X_ppp, cont= trns_y, n_estimators=1000, cv=10, max_samples=1.,
                       pctEmbargo=0, scoring='accuracy', method='SFI', minWLeaf=0.)

    end = time.time()
    print((end-start)/60)
    print(f'SFI takes {(end-start)/60/60} hours')
    
else:  
    SFI_imp_Xppp = pd.read_csv('SFI_imp_Xppp.csv', index_col = [0])
    SFI_oob_oos_Xppp = pd.read_csv('SFI_oob_oos_Xppp.csv', index_col = [0])
    SFI_oob_Xppp = SFI_oob_oos_Xppp['SFI_oob'].values[0]
    SFI_oos_Xppp = SFI_oob_oos_Xppp['SFI_oos'].values[0]

In [ ]:
# Let's set the importance measures in descending order:
imp_Xppp_cols = ['mean', 'std']
MDI_imp_Xppp.columns = imp_Xppp_cols
MDA_imp_Xppp.columns = imp_Xppp_cols
SFI_imp_Xppp.columns = imp_Xppp_cols

MDI_imp_Xppp_sorted = MDI_imp_Xppp.sort_values(by=['mean'], ascending=False)
MDA_imp_Xppp_sorted = MDA_imp_Xppp.sort_values(by=['mean'], ascending=False)
SFI_imp_Xppp_sorted = SFI_imp_Xppp.sort_values(by=['mean'], ascending=False)

In [ ]:
# Creating a DataFrame with importance measures sorted:

features_imp_Xppp = (pd.DataFrame()
                     .assign(MDI_features = MDI_imp_Xppp_sorted.index)
                     .assign(MDI_imp = MDI_imp_Xppp_sorted['mean'].values)
                     .assign(MDA_features = MDA_imp_Xppp_sorted.index)
                     .assign(MDA_imp = MDA_imp_Xppp_sorted['mean'].values)
                     .assign(SFI_features = SFI_imp_Xppp_sorted.index)
                     .assign(SFI_imp = SFI_imp_Xppp_sorted['mean'].values)
                    )
                     

In [ ]:
features_imp_Xppp.head(10)

**8.3.c) Do you appreciate significant changes in the rankings of important features, relative to the results from exercise 2?**

Among the 10 most important features issued for the three methods, the results from the previous dataset ($\ddot{X},y$) showed 3 coincidences out of 10 between MDI and MDA, and for the dataset ($\dddot{X},y$) the coincidences rose up to 9 out of 10.

Regarding the matches between MDI and SFI they fell from 8 to 5 out of 10. It is worth to note that all three methods share the first 4 most important features.

Despite most of informative features were dropped in order to generate the set X°°°, the only two informative features I_2 and I_7 were selected by MDI and MDA methods, for the case of SFI I_2 was not selected among the ten most important.

**8.4)Using the code presented in Section 8.6:**

**8.4.a) Generate a dataset (X,y) of 1E6 observations, where 5 features are imformative, 5 are redundant and 10 are noise.**

When generating the 1E6 observations with the function $getTestData()$ it issues an error in the statement that creates the index with the method $DatetimeIndex()$. It seems that the function allows certain amount of datetime elements to be created (about 70.000 under the BusinessDay frequency), then a suggested modification to solve this contrariness is to change the frequence from BusinessDay to 15 minutes periods:

$df0=pd.DatetimeIndex(periods=$n_samples$,freq=pd.tseries.offsets.Minute(30), end=pd.datetime.today())$


In [ ]:
n_datasets = 10
n_samples_8 = 1000000
n_features_8 = 20
n_informative_8 = 5
n_redundant_8 = 5

X8, y8 = getTestData_largeSet(n_features=n_features_8, n_informative=n_informative_8, 
                                  n_redundant=n_redundant_8, n_samples=n_samples_8)


**8.4.b) Split (X,y) into 10 datasets {($X_{i}$,$y_{i}$)}$_{i=1,....,10}$, each of 1E5 observations.** 

The datasets come already ordered that way from the exercise (8.4.a)

In [ ]:
X8_datasets = []
y8_datasets = []

In [ ]:
ind = np.array_split(np.arange(n_samples_8),10)

In [ ]:
ind[0]

In [ ]:
for i in range(len(ind)):
    X8_datasets.append(X8.iloc[ind[i],:])
    y8_datasets.append(y8.iloc[ind[i],:])

In [ ]:
print(f'Number of observations of dataset [0]: {X8_datasets[0].shape}')
print(f'Number of observations of dataset [9]: {X8_datasets[9].shape}')

**8.4.c) Compute the parallelized feature importance (Section 8.5), on each of the 10 datasets, {($X_{i}$,$y_i$)}$_{i=1,....10}$.**

In [ ]:
cv = 10
pctEmbargo = 0
max_samples = 1.

In [ ]:
# The following processing takes more than 9 hours just calculating MDI without a parallel process,
# then in order to skip it, the variable "processing" is set to False 
# heading off to a cell ahead where a DataFrame with the results sorted are uploaded.

#(The multiprocess is covered in chapter 20)

processing = False 

if processing:
    start = time.time()

    MDI_imp_dict = {}
    MDI_oob_dict = {}
    MDI_oos_dict = {}

    for i in range(len(X8_datasets)):
        MDI_imp, MDI_oob, MDI_oos = featImportance_2_mc(trnsX=X8_datasets[i], cont= y8_datasets[i], n_estimators=1000, 
                                                        cv=cv, max_samples=max_samples,pctEmbargo=pctEmbargo, 
                                                        scoring='accuracy', method='MDI', minWLeaf=0.)
        MDI_imp_dict['MDI_imp_'+str(i)] = MDI_imp
        MDI_oob_dict['MDI_oob_'+str(i)] = MDI_oob
        MDI_oos_dict['MDI_oos_'+str(i)] = MDI_oos

        print(f'iteracion {i}')

    end = time.time()
    print(f'Parallelized feature importance calculation takes {(end-start)/60/60} hours')
    


In [ ]:
# The code in this cell sets the results of the "MDI_imp_dict" in DataFrames, one for the column 'mean' which the 
# MDI value and other for the column 'std'.
# As the results were already carried out and saved, the staments are within a conditional
# that allows to skip them.

if processing == True:

    
    MDI_keys = list(MDI_imp_dict.keys())
    MDI_imp_mean_ALL = pd.DataFrame(index=MDI_imp_dict[MDI_keys[0]].index)
    MDI_imp_std_ALL = pd.DataFrame(index=MDI_imp_dict[MDI_keys[0]].index)
        
    count = 0

    for key in MDI_imp_dict:
        MDI_imp_mean_df = pd.DataFrame()
        MDI_imp_mean_df = MDI_imp_dict[key]
        MDI_imp_mean_ALL['mean_'+str(count)] = MDI_imp_mean_df['mean'].values
        
        MDI_imp_std_df = pd.DataFrame()
        MDI_imp_std_df = MDI_imp_dict[key]
        MDI_imp_std_ALL['std_'+str(count)] = MDI_imp_std_df['std'].values
        
        count += 1
        
else:
    MDI_imp_mean_ALL = pd.read_csv('MDI_imp_parallelized.csv', index_col = [0])

In [ ]:
# This cell orders the content of the dictionary with oob and oos values into a DataFrame.
# The content can be loaded setting "processing = False"

if processing:

    MDI_oob = []
    MDI_oos = []

    for key, value in MDI_oob_dict.items():
        MDI_oob.append(value)

    for key, value in MDI_oos_dict.items():
        MDI_oos.append(value)

    MDI_oob_oos = pd.DataFrame({'MDI_oob':MDI_oob, 'MDI_oos':MDI_oos})

else:
    MDI_oob_oos = pd.read_csv('MDI_oob_oos_parallelized.csv', index_col = [0])

In [ ]:
# Getting the global value of MDI:

In [ ]:
MDI_imp_mean_ALL['sum'] = MDI_imp_mean_ALL.sum(axis=1)

In [ ]:
MDI_imp_mean_ALL_sorted = MDI_imp_mean_ALL.sort_values(by=['sum'], ascending=False)

In [ ]:
MDI_imp_mean_ALL_sorted.head(10)

**8.4.d) Compute the stacked feature importance on the combined dataset (X,y).**

As a distributional homogeneity is inherent across all datasets {($\tilde{X}_{i}$,$y{i}$)}$_{i=1,...,10}$ due to its generation methodology, no transformation is required.

In [ ]:
# The calculation process without parallelization takes about 15 hours so the conditional is created to upload the 
# results previously calculated.

processing = False

if processing:
    start = time.time()
    MDI_imp_stacked, MDI_oob_stacked, MDI_oos_stacked = featImportance_2_mc(trnsX=X8, cont= y8, n_estimators=1000, 
                                                            cv=cv, max_samples=max_samples,pctEmbargo=pctEmbargo, 
                                                            scoring='accuracy', method='MDI', minWLeaf=0.)

    end = time.time()
    print(f'The calculation of the MDI for the combined dataset takes {(end-start)/60/60} hours')

else:
    MDI_imp_stacked = pd.read_csv('MDI_imp_stacked.csv', index_col = [0])
    MDI_oob_oos_stacked = pd.read_csv('MDI_oob_oos_stacked.csv', index_col = [0])
    MDI_oob_stacked = MDI_oob_oos_stacked.iloc[0,0]
    MDI_oos_stacked = MDI_oob_oos_stacked.iloc[1,0]

In [ ]:
MDI_imp_stacked_sorted = MDI_imp_stacked.sort_values(by = ['mean'], ascending = False)

In [ ]:
MDI_imp_stacked_sorted.head(10)

**8.4.e) What causes the discrepancy between the two?$\quad$ Which one is more reliable?**

Regarding the discrepancy between the results from the parellelzed methodology and the stacked methodology, the 10 most important features coincide in both cases. The order is not exactly the same but the same 10 features are selected in both methodologies.

With respect to the relibility of the results, as they coincide, the stacked methodology is more reliable due to the following advantages as it is mentioned in AFML section 8.5:

1)The classifier is fitted on a much larger dataset.

2)The importance is derived directly, and no weighting scheme is required for combining the results.

3)Conclusions are more general and less biased by outliers or overfitting.

4)Because importance scores are not averaged across instruments, substitution effects do not cause the dampening of those scores.

**8.5) Repeat all MDI calculations from exercises 1-4, but this time allow for masking effects. That means, do not set $max$_$features=int(1)$ in Snippet 8.2.$\quad$How do results differ as a consequence of this change? $\quad$Why?**

In [ ]:
# SNIPPET 8.8 CALLING FEATURE IMPORTANCE FOR ANY METHOD - Allowing masking effects:

   # It is the same function used to calculate feature importance but it includes a modification.
   # The parameter "max_features" within the methods "DecisionTreeClassifier()" is set to "None", which is equivalent 
   # to set max_features = n_features.
    

def featImportance_2_mc_msk(trnsX,cont,n_estimators=1000,cv=10,max_samples=1.,
                   pctEmbargo=0,scoring='accuracy',method='SFI',minWLeaf=0.,**kargs):
    
    # feature importance from a random forest
    from sklearn.tree import DecisionTreeClassifier
    from sklearn.ensemble import BaggingClassifier
    #from mpEngine import mpPandasObj                  #mc
    
    # n_jobs=(-1 if numThreads>1 else 1) # run 1 thread with ht_helper in dirac1    ...#mc
    
    #1) prepare classifier,cv. max_features=1, to prevent masking
    
    clf=DecisionTreeClassifier(criterion='entropy',max_features=None,class_weight='balanced',   
                               min_weight_fraction_leaf=minWLeaf)                   # mc
    
    #clf=BaggingClassifier(base_estimator=clf,n_estimators=n_estimators,            ...#mc
    #                        max_features=1.,max_samples=max_samples,oob_score=True,n_jobs=n_jobs)
    
    #To include making effects as required in the exercise 8.5, the parameter max_features of the BaggingClassifier is 
    # is set to 1. according to:
    
    #            max_features = If float, then draw max_features * X.shape[1] features
    
    clf=BaggingClassifier(base_estimator=clf,n_estimators=n_estimators,max_features=1.,
                          max_samples=max_samples,oob_score=True)
    fit=clf.fit(X=trnsX,y=cont['bin'],sample_weight=cont['w'].values)
    oob=fit.oob_score_
    
    if method=='MDI':
        imp=featImpMDI(fit,featNames=trnsX.columns)
        
        #oos=cvScore(clf,X=trnsX,y=cont['bin'],cv=cv,sample_weight=cont['w'],                     #original
        #            t1=cont['t1'],pctEmbargo=pctEmbargo,scoring=scoring).mean()
        
        oos = cvScore2_mc(clf,X=trnsX,y=cont['bin'],cv=cv,sample_weight=cont['w'],                # mc
                          t1=cont['t1'],pctEmbargo=pctEmbargo,scoring=scoring).mean()
              
    elif method=='MDA':
        imp,oos=featImpMDA(clf,X=trnsX,y=cont['bin'],cv=cv,sample_weight=cont['w'],
                           t1=cont['t1'],pctEmbargo=pctEmbargo,scoring=scoring)
    
    elif method=='SFI':
        cvGen=PurgedKFold(n_splits=cv,t1=cont['t1'],pctEmbargo=pctEmbargo)
        
        #oos=cvScore(clf,X=trnsX,y=cont['bin'],sample_weight=cont['w'],scoring=scoring,           #original
        #            cvGen=cvGen).mean()
        
        oos = cvScore2_mc(clf, X=trnsX,y=cont['bin'],sample_weight=cont['w'],scoring=scoring,     #...mc
                                cvGen=cvGen).mean()
        
        #clf.n_jobs=1 # paralellize auxFeatImpSFI rather than clf                                 # original
        #imp=mpPandasObj(auxFeatImpSFI,('featNames',trnsX.columns),numThreads,
        #                clf=clf,trnsX=trnsX,cont=cont,scoring=scoring,cvGen=cvGen)
        
        
        imp = auxFeatImpSFI(featNames=trnsX.columns, clf=clf, trnsX=trnsX, cont=cont, scoring=scoring, cvGen=cvGen)     #mc
        
        
    return imp,oob,oos

In [ ]:
# 8.5 - 8.1 Using the code presented in Section 8.6:

# 8.5 - 8.1(a) Generate a dataset (X, y).

In [ ]:
# "msk" suffix is to denote "masking effects"

X_msk, y_msk = getTestData()

In [ ]:
# 8.5 - 8.1(b) Apply a PCA transformation on X, which we denote ̇X.

Xp_msk = orthoFeats(X_msk, varThres = 0.95)

In [ ]:
Xp_msk.head()

In [ ]:
# 8.5 - 8.1(c) Compute MDI feature importance on (̇X, y), where the base estimator is RF. (just MDI)

# MDI:

cv = 10
pctEmbargo = 0
max_samples = 1.

calculate_MDI_msk = False

if calculate_MDI_msk:

    start = time.time()

    MDI_imp_msk, MDI_oob_msk, MDI_oos_msk = featImportance_2_mc_msk(trnsX=Xp_msk, cont= y_msk, n_estimators=1000, cv=cv, max_samples=max_samples,
                       pctEmbargo=pctEmbargo, scoring='accuracy', method='MDI', minWLeaf=0.)

    end = time.time()

    print(f'MDI_msk takes {(end-start)/60} minutes')
    
else:
    MDI_imp_msk = pd.read_csv('MDI_imp_msk.csv', index_col=[0])
    MDI_oob_oos_msk = pd.read_csv('MDI_oob_oos_msk.csv', index_col = [0])
    MDI_oob_msk = MDI_oob_oos_msk.iloc[0,0]
    MDI_oos_msk = MDI_oob_oos_msk.iloc[1,0]

In [ ]:
# (d) Do the three methods agree on the important features?  Why?

# In this case the question would be, do the MDI results calculated not allowing mask effects agree with the 
# MDI results allowing mask effects?

MDI_imp_msk_sorted = MDI_imp_msk.sort_values(by=['mean'], ascending = False)
MDI_imp_sorted = MDI_imp.sort_values(by = ['mean'], ascending = False)

MDI_imp_results = (pd.DataFrame()
                  .assign(MDI_feat = MDI_imp_sorted.index)
                  .assign(MDI = MDI_imp_sorted['mean'].values)
                  .assign(MDI_msk_feat = MDI_imp_msk_sorted.index)
                  .assign(MDI_msk = MDI_imp_msk_sorted['mean'].values))

In [ ]:
MDI_imp_results.head(10)

Observing the 10 most important features selected for both methodolgies, there is a coincide in 9 out of 10. It seems the masking effects that consist on systematically ignore some features by the tree classifiers did not take place or it took place in both cases. Let's see the following results where the calculations are over the principal components.

The modifications in order to perform the results allowing the mask effects consist in setting the parameter (max_features=None) to the classifier clf=DecisionTreeClassifier( ....,..., max_features=None,...) and the same parameter (max_features = 1.)(1. is a float number) to the classifier clf=BaggingClassifier(...,...,max_features=1.,...).

In [ ]:
# 8.5 - 8.2) From exercise 1, generate a new dataset (Ẍ , y), where Ẍ is a feature union of X and ̇X.

Xpp_msk = pd.concat([X_msk, Xp_msk], axis = 1)

In [ ]:
Xpp_msk.head()

In [ ]:
Xpp_msk.shape

In [ ]:
# 8.5 - 8.2.a) Compute MDI, MDA, and SFI feature importance on (Ẍ , y), where the base estimator is RF.
#              (Compute MDI feature importance on (Ẍ,𝑦) , where the base estimator is RF.)

# MDI for Xpp_msk:

cv = 10
pctEmbargo = 0
max_samples = 1.

calculate_MDI_msk_Xpp = False

if calculate_MDI_msk_Xpp:

    start = time.time()

    MDI_imp_msk_Xpp, MDI_oob_msk_Xpp, MDI_oos_msk_Xpp = featImportance_2_mc_msk(trnsX=Xpp_msk, cont= y_msk, n_estimators=1000, cv=cv, max_samples=max_samples,
                       pctEmbargo=pctEmbargo, scoring='accuracy', method='MDI', minWLeaf=0.)

    end = time.time()

    print(f'MDI_msk_Xpp takes {(end-start)/60} minutes')
    
else:
    MDI_imp_msk_Xpp = pd.read_csv('MDI_imp_msk_Xpp.csv', index_col=[0])
    MDI_oob_oos_msk_Xpp = pd.read_csv('MDI_oob_oos_msk_Xpp.csv', index_col = [0])
    MDI_oob_msk_Xpp = MDI_oob_oos_msk_Xpp.iloc[0,0]
    MDI_oos_msk_Xpp = MDI_oob_oos_msk_Xpp.iloc[1,0]
    

In [ ]:
# 8.5 - 8.2.b) Do the three methods agree on the important features? Why? 
#              In this case the question would be: 
#              Do the MDI results allowing mask effects calculated upon the X' agree to the MDI (mask effects allowed)
#              results calculated on X?
# 
#              Other interesting question could be: Do the MDI results allowing mask effects calculated upon the X'
#              agree to the MDI without mask effects (upon the same dataset X')?

MDI_imp_sorted
MDI_imp_msk_Xpp_sorted = MDI_imp_msk_Xpp.sort_values(by=['mean'], ascending = False)

MDI_msk_Xp_Xpp = (pd.DataFrame()
                  .assign(Xp_features = MDI_imp_sorted.index)
                  .assign(MDI_Xp = MDI_imp_sorted['mean'].values)
                  .assign(Xpp_features = MDI_imp_msk_Xpp_sorted.index[:len(MDI_imp_sorted.index)])
                  .assign(MDI_Xpp = MDI_imp_msk_Xpp_sorted['mean'].values[:len(MDI_imp_sorted.index)]))


In [ ]:
MDI_msk_Xp_Xpp.head(10)

The *Xp_features* are integers because they refer to PCA that comprise the 95% of the variation of the original data set, while the *Xpp_features* include the name of certain features because the dataset X°° is composed by the aforementioned PCA and the original features of the original dataset.

The results of MDI allowing masking effects calculated over the X and X°° datasets agree only on one feature, the first principal component.

In [ ]:
# Do the MDI results allowing mask effects calculated upon the X° agree with 
# the MDI without mask effects (upon the same dataset X°)?

In [ ]:
MDI_imp_msk_Xpp_sorted = MDI_imp_msk_Xpp.sort_values(by=['mean'], ascending = False)

In [ ]:
MDI_Xpp_comparison_results = (pd.DataFrame()
                  .assign(features_Xpp = MDI_imp_Xpp_sorted['features'])
                  .assign(MDI_Xpp = MDI_imp_Xpp_sorted['mean'].values)
                  .assign(features_msk = MDI_imp_msk_Xpp_sorted.index)
                  .assign(MDI_Xpp_msk = MDI_imp_msk_Xpp_sorted['mean'].values))

In [ ]:
MDI_Xpp_comparison_results.head(10)

Within the most important selected features there are coincidences in 6 out of 10 features. The masking effects become more evident.

In [ ]:
# 8.5 - 8.3) Take the results from exercise 2:

In [ ]:
# 8.5 - 8.3.a) Drop the most important features according to each method (MDI in this exercise), 
# resulting in a features matrix  X°°°.

In [ ]:
MDI_msk_Xp_Xpp.head(10)

In [ ]:
Xp_msk_topFeatures = list(MDI_msk_Xp_Xpp['Xp_features'].head(10).values)
print(f'Xp_msk_topFeatures: {Xp_msk_topFeatures}')

Xpp_msk_topFeatures = list(MDI_msk_Xp_Xpp['Xpp_features'].head(10).values)
print(f'\nXpp_msk_topFeatures: {Xpp_msk_topFeatures}')

In [ ]:
Xpp_msk_features_drop = Xp_msk_topFeatures + Xpp_msk_topFeatures

In [ ]:
# Some features stored in Xppp_msk_features_all have int() format as a consecuence of the orthogonalization (PCA calculation), 
# so it is necessary to transform them to str() format.

Xpp_features_str = []
for f in Xpp_msk_features_drop:
    Xpp_features_str.append(str(f))

Xpp_msk_features_drop = Xpp_features_str




# Remove duplicates:

Xpp_msk_features = []

for feat in Xpp_msk_features_drop:
    if feat not in Xpp_msk_features:
        Xpp_msk_features.append(feat)

In [ ]:
# Converting all the columns names to strings:
Xpp_cols = []
for col in Xpp_msk.columns:
    Xpp_cols.append(str(col))

Xpp_msk.columns = Xpp_cols
    

In [ ]:
Xppp_msk = Xpp_msk.drop(Xpp_msk_features_drop, axis=1)

In [ ]:
Xppp_msk.head()

In [ ]:
#8.5. - 8.3.b) Compute MDI feature importance on (X°°°, y), where the base estimator is RF.

# MDI for Xpp_msk:

cv = 10
pctEmbargo = 0
max_samples = 1.

calculate_MDI_msk_Xppp = False

if calculate_MDI_msk_Xppp:

    start = time.time()

    MDI_imp_msk_Xppp, MDI_oob_msk_Xppp, MDI_oos_msk_Xppp = featImportance_2_mc_msk(trnsX=Xppp_msk, cont= y_msk, n_estimators=1000, cv=cv, max_samples=max_samples,
                       pctEmbargo=pctEmbargo, scoring='accuracy', method='MDI', minWLeaf=0.)

    end = time.time()

    print(f'MDI_msk_Xpp takes {(end-start)/60} minutes')
    
    MDI_imp_msk_Xppp.to_csv('MDI_imp_msk_Xppp.csv')
    
    MDI_oob_oos_msk_Xppp = pd.DataFrame({'values':[MDI_oob_msk_Xppp, MDI_oos_msk_Xppp]}, index=['MDI_oob','MDI_oos'])
    MDI_oob_oos_msk_Xppp.to_csv('MDI_oob_oos_msk_Xppp.csv')
    
else:
    MDI_imp_msk_Xppp = pd.read_csv('MDI_imp_msk_Xppp.csv', index_col=[0])
    MDI_oob_oos_msk_Xppp = pd.read_csv('MDI_oob_oos_msk_Xppp.csv', index_col = [0])
    MDI_oob_msk_Xppp = MDI_oob_oos_msk_Xppp.iloc[0,0]
    MDI_oos_msk_Xppp = MDI_oob_oos_msk_Xppp.iloc[1,0]
    

In [ ]:
MDI_imp_msk_Xppp_sorted = MDI_imp_msk_Xppp.sort_values(by=['mean'], ascending = False)

In [ ]:
MDI_imp_msk_Xppp_sorted.head()

In [ ]:
# 8.5 - 8.3.c) Do you appreciate significant changes in the rankings of important features, relative to the results 
#           from exercise 2?

In [ ]:
MDI_imp_msk_Xpp_sorted.shape[0]

In [ ]:
MDI_Xpp_Xppp_results = (pd.DataFrame()
                       .assign(Xpp_msk_feat = MDI_imp_msk_Xpp_sorted.index[:MDI_imp_msk_Xppp_sorted.shape[0]])
                       .assign(MDI_Xpp = MDI_imp_msk_Xpp_sorted['mean'].values[:MDI_imp_msk_Xppp_sorted.shape[0]])
                       .assign(Xppp_msk_feat = MDI_imp_msk_Xppp_sorted.index)
                       .assign(MDI_Xppp = MDI_imp_msk_Xppp_sorted['mean'].values))

In [ ]:
MDI_Xpp_Xppp_results.head(10)

There are not coincidences among the first 10 most important features selected for both datasets, remember that the X°°° dataset is generated droping the most important features in X°°. Neverthelesss the features are quite similar regarding its kind, for instance the informative features selected in the X°° dataset are close to the informative features selected in the X°°° dataset, let's see:

*X°°=(I_0, I_2, I_4, I_6, I_8, I_9)*

*X°°°=(I_1, I_3, I_5, I_7)*

Among both selected features sets are the informative ones which is coherent with the objective of the exercise. 
It is worth to mention that in spite of the mask effects allowed in this exercise the selected features were the most important. 

In [ ]:
# 8.5 - 8.4) Using the code presented in Section 8.6:

In [ ]:
# 8.5 - 8.4.a) Generate a dataset (X,y) of 1E6 observations, where 5 features are informative, 5 are redundant and 10 are noise.

n_datasets = 10
n_samples_8 = 1000000
n_features_8 = 20
n_informative_8 = 5
n_redundant_8 = 5

Xmsk_e8, ymsk_e8 = getTestData_largeSet(n_features=n_features_8, n_informative=n_informative_8, 
                                  n_redundant=n_redundant_8, n_samples=n_samples_8)


In [ ]:
print(f'Xmsk_e8.shape: {Xmsk_e8.shape}')
print(f'ymsk_e8.shape: {ymsk_e8.shape}')

In [ ]:
# 8.5 - 8.4.b) Split (X, y) into 10 datasets {(Xi, yi)}i=1,…,10, each of 1E5 observations.

indices = np.array_split(np.arange(Xmsk_e8.shape[0]),10)

In [ ]:
X_datasets_msk = []
y_datasets_msk = []

for i in indices:
    X_datasets_msk.append(Xmsk_e8.iloc[i[0]:i[-1]+1,:])
    y_datasets_msk.append(ymsk_e8.iloc[i[0]:i[-1]+1,:])
    

In [ ]:
# 8.5 - 8.4.c) Compute the parallelized feature importance (Section 8.5), on each of the 10
# datasets, {(Xi, yi)}i=1,…,10.


# MDI for X_datasets_msk:

cv = 10
pctEmbargo = 0
max_samples = 1.

calculate_MDI_msk_85_84c = False

if calculate_MDI_msk_85_84c:

    start_global = time.time()
    MDI_msk_85_84c_list = []
    
    for i in range(len(X_datasets_msk)):
        print(f'iteration {i}')
        start_i = time.time()

        MDI_imp_msk_85_84c, MDI_oob_msk_85_84c, MDI_oos_msk_85_84c = featImportance_2_mc_msk(trnsX=X_datasets_msk[i], cont= y_datasets_msk[i], n_estimators=1000, cv=cv, max_samples=max_samples,
                           pctEmbargo=pctEmbargo, scoring='accuracy', method='MDI', minWLeaf=0.)
        
        MDI_msk_85_84c_list.append(MDI_imp_msk_85_84c)
        end_i = time.time()
        
        print(f'elapsed time: {(end_i-start_i)/60} minutes')

    end_global = time.time()

    print(f'All MDI_msk_85_84c takes {(end_global-start_global)/60} minutes')
    
    MDI_msk_85_84c = (pd.DataFrame()
                 .assign(features_1 = MDI_msk_85_84c_list[0].sort_values(by=['mean'], ascending=False).index)
                 .assign(MDI_1 = MDI_msk_85_84c_list[0].sort_values(by=['mean'], ascending=False)['mean'].values)
                 .assign(features_2 = MDI_msk_85_84c_list[1].sort_values(by=['mean'], ascending=False).index)
                 .assign(MDI_2 = MDI_msk_85_84c_list[1].sort_values(by=['mean'], ascending=False)['mean'].values)
                 .assign(features_3 = MDI_msk_85_84c_list[2].sort_values(by=['mean'], ascending=False).index)
                 .assign(MDI_3 = MDI_msk_85_84c_list[2].sort_values(by=['mean'], ascending=False)['mean'].values) 
                 .assign(features_4 = MDI_msk_85_84c_list[3].sort_values(by=['mean'], ascending=False).index)
                 .assign(MDI_4 = MDI_msk_85_84c_list[3].sort_values(by=['mean'], ascending=False)['mean'].values)
                 .assign(features_5 = MDI_msk_85_84c_list[4].sort_values(by=['mean'], ascending=False).index)
                 .assign(MDI_5 = MDI_msk_85_84c_list[4].sort_values(by=['mean'], ascending=False)['mean'].values)
                 .assign(features_6 = MDI_msk_85_84c_list[5].sort_values(by=['mean'], ascending=False).index)
                 .assign(MDI_6 = MDI_msk_85_84c_list[5].sort_values(by=['mean'], ascending=False)['mean'].values)
                 .assign(features_7 = MDI_msk_85_84c_list[6].sort_values(by=['mean'], ascending=False).index)
                 .assign(MDI_7 = MDI_msk_85_84c_list[6].sort_values(by=['mean'], ascending=False)['mean'].values)
                 .assign(features_8 = MDI_msk_85_84c_list[7].sort_values(by=['mean'], ascending=False).index)
                 .assign(MDI_8 = MDI_msk_85_84c_list[7].sort_values(by=['mean'], ascending=False)['mean'].values)
                 .assign(features_9 = MDI_msk_85_84c_list[8].sort_values(by=['mean'], ascending=False).index)
                 .assign(MDI_9 = MDI_msk_85_84c_list[8].sort_values(by=['mean'], ascending=False)['mean'].values)
                 .assign(features_10 = MDI_msk_85_84c_list[9].sort_values(by=['mean'], ascending=False).index)
                 .assign(MDI_10 = MDI_msk_85_84c_list[9].sort_values(by=['mean'], ascending=False)['mean'].values)
                 )
    
    MDI_msk_85_84c.to_csv('MDI_msk_85_84c.csv')
    
    
else: 
    MDI_msk_85_84c = pd.read_csv('MDI_msk_85_84c.csv', index_col = [0])


In [ ]:
MDI_msk_85_84c.head()

In [ ]:
# 8.5 - 8.4.d) Compute the stacked feature importance on the combined dataset (X, y).

# MDI for the stacked dataset_msk:

cv = 10
pctEmbargo = 0
max_samples = 1.

calculate_MDI_msk_85_84d = True

if calculate_MDI_msk_85_84d:

    start_global = time.time()
    
    MDI_imp_msk_85_84d, MDI_oob_msk_85_84d, MDI_oos_msk_85_84d = featImportance_2_mc_msk(trnsX=Xmsk_e8, cont= ymsk_e8, n_estimators=1000, cv=cv, max_samples=max_samples,
                           pctEmbargo=pctEmbargo, scoring='accuracy', method='MDI', minWLeaf=0.)

    end_global = time.time()

    print(f'All MDI_msk_85_84d takes {(end_global-start_global)/60} minutes')
    
    MDI_imp_msk_85_84d_sorted = MDI_imp_msk_85_84d.sort_values(by=['mean'], ascending=False) 
    
    MDI_imp_msk_85_84d_sorted.to_csv('MDI_imp_msk_85_84d.csv')

else:
    MDI_imp_msk_85_84d = pd.read_csv('MDI_imp_msk_85_84d.csv', index_col = [0])
    